In [39]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import warnings
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse, r2_score
from typing import Tuple
from abc import ABC, abstractmethod
from collections import deque
from typing import Tuple
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from tqdm.notebook import tqdm, trange
from sklearn.preprocessing import Normalizer

# Домашняя работа: ансамбли

Эта домашняя работа является идейным продолжением предыдущей. В данной работе вам будет необходимо поэкспериментировать с различными методами ансамблирования и проверить, какие из них работают лучше. Пайплайн предобработки данных можно взять полностью из предыдущей работы.

Требования к домашней работе:
- Во всех графиках (если вы их строите) должны быть подписи через title, legend, etc.
- Во время обучения моделей проверяйте, что у вас не текут данные. Обычно это позитивно влияет на качество модели на тесте, но негативно влияет на оценку 🌚
- Если вы сдаете работу в Google Colaboratory, убедитесь, что ваша тетрадка доступна по ссылке. Если в итоге по каким-то причинам тетрадка не будет открываться у преподавателя, задание не будет засчитано
- Использование мемов допускается. Если задания дались тяжело, можно дополнительно приложить какой-нибудь постироничный мем про ваши страдания во время выполнения данной домашней работы. За мемы с использованием нецензурной лексики баллы будут снижены.

# Загрузка и подготовка данных (1 балл)

В этой секции предлагается прогнать предобработку данных из прошлой тетрадки заново и сохранить получившийся датасет в формате csv.

Если вы **не хотите заморачиваться**, то просто скопируйте код с предобработкой ниже.

В противном случае в старой тетрадке:
1. Отдельно выполните предобработку (`fit_transform`) тренировочной части данных
2. Добавьте колонку `split` к датафрейму с обучающей выборкой, в этой колонке проставьте значение `train` для всех объектов
3. Затем примените **только** предобработку (`transform`) к тестовой части данных
4. Добавьте колонку `split` к тестовой выборке, в этой колонке проставьте значение `test` для всех объектов
5. Объедините два датафрейма в один при помощи функции `pd.concat`
6. Сохраните получившийся датафрейм при помощи функции `to_csv`, не забудьте передать аргумент `index=False`

Получившийся файл сохраните отдельно и используйте в этой домашней работе. Для разбиения датасета на обучающую и тестовую части вместо функции `train_test_split` можете применять колонку `split`.

In [40]:
# !pip install gdown
# !gdown 18PVwZWFbpRbEHW-Hc8R0DUTl9CF1aQa0 -O data.csv

In [41]:
df = pd.read_csv('data.csv').drop(columns=[
    'product_name',
    'index',
    'uniq_id',
    'customers_who_bought_this_item_also_bought',
    'items_customers_buy_after_viewing_this_item',
    'sellers',
    'description', # text
    'product_information', # text
    'product_description', # text
    'customer_questions_and_answers', # text
    'customer_reviews', # text
])

### Посмотрим на данные

In [42]:
df.head()

,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category
0,Hornby,£3.42,5 new,15,1.0,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...
1,FunkyBuys,£16.99,NaN,2,1.0,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...
2,ccf,£9.99,2 new,17,2.0,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...
3,Hornby,£39.99,NaN,1,2.0,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...
4,Hornby,£32.19,NaN,3,2.0,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...


### Количество пропусков

In [43]:
df.isna().sum()

manufacturer                           5
price                                  0
number_available_in_stock           2211
number_of_reviews                     15
number_of_answered_questions         673
average_review_rating                 15
amazon_category_and_sub_category     550
dtype: int64

### Процент пропусков

In [44]:
df.isna().sum() / df.shape[0] * 100

manufacturer                         0.058500
price                                0.000000
number_available_in_stock           25.868726
number_of_reviews                    0.175500
number_of_answered_questions         7.874108
average_review_rating                0.175500
amazon_category_and_sub_category     6.435006
dtype: float64

### Заполнение пропусков
**Поля**
1. manufacturer - заполним "unknown" на следующем шаге и переведем в категориальный признак
2. number_available_in_stock - количество_отвеченных_вопросов, заполним пропуски 0 на следующем шаге
3. number_of_reviews - количество просмотров, заполним 0 на следующем шаге
4. number_of_answered_questions - количество_отвеченных_вопросов, заполним 0 на следующем шаге
5. average_review_rating - усреднение_просмотров_рейтинг, удалим строки с пропусков в данной колонке
6. amazon_category_and_sub_category - амазон_категория_и_суб_категория, заполним "unknown" и переведем в категориальный

Заполнение 0 или "unknown" происходит, потому что:
* в количественных столбцах пропуски означают отсутствие данного признака, следовательно, можем использовать 0
* в категориальных заполним константой "unknown"

In [45]:
df['manufacturer'] = df['manufacturer'].fillna("unknown")
df['number_available_in_stock'] = df['number_available_in_stock'].fillna(0)
df['number_of_reviews'] = df['number_of_reviews'].fillna(0)
df['number_of_answered_questions'] = df['number_of_answered_questions'].fillna(0)
df['amazon_category_and_sub_category'] = df['amazon_category_and_sub_category'].fillna("unknown")

df = df.dropna()

## Подготовка данных
### Работа с числовыми данными
Переведем price в числовой тип данных

In [46]:
df['price'] = df['price'].str.replace('£', '')
df['price'] = df['price'].str.replace(',', '')
df['price'] = df['price'].astype('float')

Переведем number_available_in_stock в числовой тип данных

In [47]:
df['number_available_in_stock'] = df['number_available_in_stock']\
    .replace('new', '', regex=True).replace('used', '', regex=True)\
    .replace('refurbished', '', regex=True).replace('collectible', '', regex=True).astype('uint8')

change data type of number_of_reviews to int

In [48]:
df['number_of_reviews'] = df['number_of_reviews'].str.replace(',', '').astype('uint16')

change data type of number_of_answered_questions to int

In [49]:
df['number_of_answered_questions'] = df['number_of_answered_questions'].astype('uint8')

change data type of average_review_rating to float

In [50]:
df['average_review_rating'] = df['average_review_rating'].replace('out of 5 stars', '', regex=True).astype('float')

### Работа с категориальными данными
cтолбец manufacturer

In [51]:
df['manufacturer'].value_counts()

Oxford Diecast        152
LEGO                  138
Disney                136
Playmobil             117
The Puppet Company    102
                     ... 
Noris                   1
Mammut                  1
M Gordon & Sons         1
Stephens                1
Green Hornet            1
Name: manufacturer, Length: 2358, dtype: int64

2358 - количество уникальных значений, прямое кодирование (One-Hot Encoding, OHE) для данного столбца не подойдет,
тк добавлять 2358 столбцов не целесообразно. Используем порядковое кодирование (Ordinal Encoding)

In [52]:
encoder = OrdinalEncoder()

df_new_ordinal = pd.DataFrame(encoder.fit_transform(df), columns=df.columns)

df['manufacturer'] = df_new_ordinal['manufacturer'].astype('float')

column amazon_category_and_sub_category

In [53]:
df['amazon_category_and_sub_category'].value_counts()

Die-Cast & Toy Vehicles > Toy Vehicles & Accessories > Scaled Models > Vehicles    784
unknown                                                                            545
Figures & Playsets > Science Fiction & Fantasy                                     359
Arts & Crafts > Children's Craft Kits > Bead Art & Jewellery-Making                340
Characters & Brands > Disney > Toys                                                292
                                                                                  ... 
Electronic Toys > Kids Remote & App Controlled Toys                                  1
Characters & Brands > Harry Potter > Books > Stationery                              1
Games > Casino Equipment > Game Sets > Roulette Sets                                 1
Games > Casino Equipment > Game Layouts > Poker Layouts                              1
Games > Drinking Games                                                               1
Name: amazon_category_and_sub_category, Len

Разобьем данный столбец на 3 столбца (иерархия категорий)

In [54]:
new_df = df['amazon_category_and_sub_category'].str.split('>', expand=True)

Закодируем каждый из полученных столбцов используя порядковое кодирование (Ordinal Encoding)

In [55]:
encoder = OrdinalEncoder()

df_new_ordinal = pd.DataFrame(encoder.fit_transform(new_df), columns=new_df.columns)

Добавим столбцы к исходному df

In [56]:
df = df.drop(columns=['amazon_category_and_sub_category'])
df = pd.concat([df, df_new_ordinal], axis=1)

Удалим оставшиеся пропуски

In [57]:
df = df.dropna()

Важно: во всех разделах ниже задачу регрессии важно оценивать не только при помощи `MSE`, но и при помощи `r2_score`. Если вы хотите перебрать какой-либо гиперпараметр, не забывайте оценивать то, насколько сильно переобучается модель и как меняется каждый из параметров в процессе обучения.

In [58]:
warnings.filterwarnings('ignore')

In [59]:
X, y = df.drop(columns=['price']), df.price
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

Нормализуем данные

In [60]:
#X_train = Normalizer().fit_transform(X_train)
#X_test = Normalizer().fit_transform(X_test)

# Стекинг (максимум 3 балла)

Решите задачу, используя разные комбинации базовых моделей. В качестве базовой модели обязательно попробуйте линейную регрессию, дерево и SVM для регрессии.

Какой набор моделей дает лучший результат? Попробуйте улучшить его, перебрав несколько гиперпараметров (как у базовой модели, так и у ансамбля).

## Простой стекинг своими руками (2 балла)

In [61]:
class StackingRegressionSolver:
    def __init__(self, base_estimators: list, meta_estimator):
        self._base_estimators = base_estimators
        self._meta_estimator = meta_estimator

    def _fit_base(self, X_train: pd.DataFrame, y_train: pd.Series) -> None:
        for estimator in tqdm(self._base_estimators):
            estimator.fit(X_train, y_train)

    def _predict_base(self, X_train: pd.DataFrame) -> pd.DataFrame:
        '''Размерность [k, N]'''
        meta_features = []
        for estimator in self._base_estimators:
            meta_features.append(estimator.predict(X_train))
        df = pd.DataFrame(meta_features)
        return df.T # Размерность [N, k]

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series) -> None:
        self._fit_base(X_train, y_train)
        meta_features = self._predict_base(X_train)
        assert meta_features.shape[0] == y_train.shape[0], 'Не совпадающие размерности'
        self._meta_estimator.fit(meta_features, y_train)

    def predict(self, X_test: pd.DataFrame) -> pd.Series:
        meta_features = self._predict_base(X_test)
        return self._meta_estimator.predict(meta_features)

In [62]:
def get_score(y_pred, y_test, title='') -> None:
    if title:
        print(title)
    print('mse =', round(mse(y_test, y_pred), 2))
    print('r2_score =', round(r2_score(y_test, y_pred), 2))

### Линейная регрессия

In [63]:
linear = LinearRegression(n_jobs=-1)
linear.fit(X_train, y_train)
y_pred = linear.predict(X_test)

get_score(y_pred, y_test)

mse = 1516.25
r2_score = 0.03


### Решающее дерево
Возьмем гипер-параметры из 9 домашки,
* max_depth = 6
* min_samples_split = 99
* min_samples_leaf = 5

In [64]:
tree = DecisionTreeRegressor(max_depth=6, min_samples_split=99, min_samples_leaf=5)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
get_score(y_pred, y_test)

mse = 1530.47
r2_score = 0.02


### Простое ансамблирование

In [65]:
class EnsembleTreeRegressor:
    def __init__(self, num_trees=100, **model_kwargs):
        self._samples_frac = 0.8
        self._trees = [DecisionTreeRegressor(**model_kwargs) for _ in range(num_trees)]

    def fit(self, x: pd.DataFrame, y: pd.Series):
        for tree in self._trees:
            tree_x = x.sample(frac=self._samples_frac, random_state=42)
            tree_y = y[tree_x.index]
            tree.fit(tree_x, tree_y)

    def predict(self, x: pd.DataFrame):
        # В качестве предсказания ансамбля будем выдавать усреднение предсказаний деревьев
        tree_y_pred = np.zeros((x.shape[0]))
        num_trees = 0
        for tree in self._trees:
            num_trees += 1
            tree_y_pred += tree.predict(x)

        return tree_y_pred / num_trees

In [66]:
ensemble = EnsembleTreeRegressor(max_depth=6, min_samples_split=99, min_samples_leaf=5)
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_test)

get_score(y_pred, y_test)

mse = 1526.44
r2_score = 0.02


In [67]:
def step(meta_model, kwargs):
    stacking_regressor = StackingRegressionSolver(
        base_estimators=[
            LinearRegression(n_jobs=-1), DecisionTreeRegressor(**kwargs),
            SVR(), EnsembleTreeRegressor(**kwargs)
        ],
        meta_estimator=meta_model
    )
    stacking_regressor.fit(X_train, y_train)
    y_pred = stacking_regressor.predict(pd.DataFrame(X_test))

    get_score(y_pred, y_test, f'meta_model = {meta_model}, kwargs = {kwargs}')

In [68]:
params = {'max_depth':6, 'min_samples_split':99, 'min_samples_leaf':5}
for meta in tqdm([LinearRegression(n_jobs=-1), DecisionTreeRegressor(), SVR()]):
    step(meta, {})
    step(meta, params)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

meta_model = LinearRegression(n_jobs=-1), kwargs = {}
mse = 2887.59
r2_score = -0.86


  0%|          | 0/4 [00:00<?, ?it/s]

meta_model = LinearRegression(n_jobs=-1), kwargs = {'max_depth': 6, 'min_samples_split': 99, 'min_samples_leaf': 5}
mse = 1530.78
r2_score = 0.02


...

meta_model = DecisionTreeRegressor(), kwargs = {'max_depth': 6, 'min_samples_split': 99, 'min_samples_leaf': 5}
mse = 2732.32
r2_score = -0.76


  0%|          | 0/4 [00:00<?, ?it/s]

meta_model = SVR(), kwargs = {}
mse = 1832.58
r2_score = -0.18


  0%|          | 0/4 [00:00<?, ?it/s]

meta_model = SVR(), kwargs = {'max_depth': 6, 'min_samples_split': 99, 'min_samples_leaf': 5}
mse = 1584.4
r2_score = -0.02


**Результаты с использованием нормализации признаков:**
* meta_model = LinearRegression(), kwargs = {}
 mse = 2837.96 r2_score = -0.82
* meta_model = LinearRegression(), kwargs = {'max_depth': 6, 'min_samples_split': 99, 'min_samples_leaf': 5}
 mse = 1631.62 r2_score = -0.05
* meta_model = DecisionTreeRegressor(), kwargs = {}
 mse = 2930.63 r2_score = -0.88
* meta_model = DecisionTreeRegressor(), kwargs = {'max_depth': 6, 'min_samples_split': 99, 'min_samples_leaf': 5}
 mse = 10046.93  r2_score = -5.46
* meta_model = SVR(), kwargs = {}  mse = 1823.18
 r2_score = -0.17
* meta_model = SVR(), kwargs = {'max_depth': 6, 'min_samples_split': 99, 'min_samples_leaf': 5}
 mse = 1593.39  r2_score = -0.02

**Результаты без использования нормализации:**
* meta_model = LinearRegression(), kwargs = {}
 mse = 2791.35 r2_score = -0.79
* meta_model = LinearRegression(), kwargs = {'max_depth': 6, 'min_samples_split': 99, 'min_samples_leaf': 5}
 mse = 1530.78 r2_score = 0.02
* meta_model = DecisionTreeRegressor(), kwargs = {}
 mse = 2958.28 r2_score = -0.9
* meta_model = DecisionTreeRegressor(), kwargs = {'max_depth': 6, 'min_samples_split': 99, 'min_samples_leaf': 5}
 mse = 2789.74 r2_score = -0.79
* meta_model = SVR(), kwargs = {}
 mse = 1837.1 r2_score = -0.18
* meta_model = SVR(), kwargs = {'max_depth': 6, 'min_samples_split': 99, 'min_samples_leaf': 5}
 mse = 1584.4 r2_score = -0.02


"Наилучшие" результаты показала (без нормализации признаков)
meta_model = LinearRegression(), kwargs = {'max_depth': 6, 'min_samples_split': 99, 'min_samples_leaf': 5}
mse = 1530.78 r2_score = 0.02

## Использование встроенной модели стекинга (0.5 балла)

In [69]:
from sklearn.ensemble import StackingRegressor, RandomForestRegressor

estimators = [
    ('lr', LinearRegression(n_jobs=-1)),
    ('svr', SVR()),
    ('dtr', DecisionTreeRegressor(**params))
]

reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(n_estimators=10, random_state=42)
)

reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

get_score(y_pred, y_test)

mse = 1961.62
r2_score = -0.26


Результаты встроенной модели оказались хуже, чем самописной

## Блендинг (0.5 балла)

Реализуйте схему блендинга. Для этого разбейте **тестовую** выборку на *валидационную* и *тестовую* части, при необходимости также доработайте код класса `StackingRegressionSolver`. Используйте для обучения базовых моделей обучающую выборку, а для обучения метамодели - валидационную.

Как изменилось качество? Как вы думаете, правдоподобнее ли выглядит такой результат?

In [70]:
train, validate, test = np.split(df.sample(frac=1, random_state=42), [int(.6*len(df)), int(.8*len(df))])

X_train, y_train = train.drop(columns=['price']), train.price
X_validate, y_validate = validate.drop(columns=['price']), validate.price
X_test, y_test = test.drop(columns=['price']), test.price

In [71]:
class BlendingRegressionSolver:
    def __init__(self, base_estimators: list, meta_estimator):
        self._base_estimators = base_estimators
        self._meta_estimator = meta_estimator

    def _fit_base(self, X_train: pd.DataFrame, y_train: pd.Series) -> None:
        for estimator in tqdm(self._base_estimators):
            estimator.fit(X_train, y_train)

    def _predict_base(self, X_valid: pd.DataFrame) -> pd.DataFrame:
        '''Размерность [k, N]'''
        meta_features = []
        for estimator in self._base_estimators:
            meta_features.append(estimator.predict(X_valid))
        df = pd.DataFrame(meta_features)
        return df.T # Размерность [N, k]

    def fit(self, X_valid: pd.DataFrame, y_valid: pd.Series,
            X_train: pd.DataFrame, y_train: pd.Series) -> None:
        self._fit_base(X_train, y_train)
        meta_features = self._predict_base(X_valid)
        assert meta_features.shape[0] == y_valid.shape[0], 'Не совпадающие размерности'
        self._meta_estimator.fit(meta_features, y_valid)

    def predict(self, X_test: pd.DataFrame) -> pd.Series:
        meta_features = self._predict_base(X_test)
        return self._meta_estimator.predict(meta_features)

In [72]:
blending_regressor = BlendingRegressionSolver(
    base_estimators=[
        LinearRegression(n_jobs=-1), DecisionTreeRegressor(**params),
        SVR(), EnsembleTreeRegressor(**params)
    ],
    meta_estimator=LinearRegression(n_jobs=-1)
)
blending_regressor.fit(X_train, y_train, X_validate, y_validate)
y_pred = blending_regressor.predict(pd.DataFrame(X_test))

get_score(y_pred, y_test)

  0%|          | 0/4 [00:00<?, ?it/s]

mse = 1630.23
r2_score = 0.02


Результаты Блендинга оказались лучше, чем результаты встроенной модели стекинга, но хуже самописной модели стэккинга.
*Как изменилось качество?* Качество практически не изменилось
*Как вы думаете, правдоподобнее ли выглядит такой результат?* Да, но меня пугает такая большая ошибка

# Бэггинг (максимум 3 балла)

В этой секции аналогично нужно решить задачу при помощи бэггинга - сначала написанного самостоятельно, а затем взятого из sklearn.

## Бэггинг своими руками (2 балла)

Решите задачу, используя в качестве базовой модели линейную регрессию, дерево и SVM. Какой из алгоритмов в качестве базовой модели дает лучший результат? Почему, как вы думаете?

In [73]:
class BaggingRegressionSolver:
    def __init__(
        self,
        base_estimator_ctor,
        max_samples: float = 1,
        n_estimators: int = 100,
        sample_random_state=42,
        **model_kwargs
    ):
        if max_samples < 0 or max_samples > 1:
            raise ValueError
        self._estimators = [
            base_estimator_ctor(**model_kwargs) for _ in range(n_estimators)
        ]
        self._max_samples = max_samples
        self._random_state = sample_random_state

    def _sample_data(self, X: pd.DataFrame, y: pd.Series) -> Tuple[pd.DataFrame, pd.Series]:
        x_i = X.sample(frac=self._max_samples, random_state=self._random_state)
        y_i = y.loc[x_i.index]
        return x_i, y_i

    def fit(self, X: pd.DataFrame, y: pd.Series):
        for estimator in self._estimators:
            x_i, y_i = self._sample_data(X, y)
            estimator.fit(x_i, y_i)

    def predict(self, X: pd.DataFrame) -> pd.Series:
        y = []
        for estimator in self._estimators:
            y_pred = estimator.predict(X)
            y.append(y_pred)
        ser = pd.DataFrame(y)
        return ser.mean()

In [74]:
for meta in tqdm([LinearRegression, DecisionTreeRegressor, SVR]):
    brs = BaggingRegressionSolver(base_estimator_ctor=meta)
    brs.fit(X_train, y_train)
    y_pred = brs.predict(X_test)

    get_score(y_pred, y_test, f'model = {meta}')

  0%|          | 0/3 [00:00<?, ?it/s]

model = <class 'sklearn.linear_model._base.LinearRegression'>
mse = 1655.88
r2_score = 0.01
model = <class 'sklearn.tree._classes.DecisionTreeRegressor'>
mse = 3890.4
r2_score = -1.33
model = <class 'sklearn.svm._classes.SVR'>
mse = 1762.32
r2_score = -0.06


## Использование встроенной модели бэггинга (1 балл)

Решите задачу, используя:
- `sklearn.ensemble.BaggingRegressor`. В качестве базовой модели попробуйте линейную регрессию, дерево и SVM
- `sklearn.ensemble.RandomForestRegressor`

Какая модель дает лучший результат? Попробуйте улучшить его, перебрав несколько гиперпараметров (как у базовой модели, так и у ансамбля).



In [77]:
from sklearn.ensemble import BaggingRegressor

for meta in tqdm([LinearRegression(), DecisionTreeRegressor(), SVR()]):
    br = BaggingRegressor(base_estimator=meta)

    br.fit(X_train, y_train)
    y_pred = br.predict(X_test)

    get_score(y_pred, y_test, f'model = {meta}, bagging = BaggingRegressor')

  0%|          | 0/3 [00:00<?, ?it/s]

model = LinearRegression(), bagging = BaggingRegressor
mse = 1653.75
r2_score = 0.01
model = DecisionTreeRegressor(), bagging = BaggingRegressor
mse = 2158.34
r2_score = -0.29
model = SVR(), bagging = BaggingRegressor
mse = 1764.76
r2_score = -0.06


In [78]:
from sklearn.ensemble import RandomForestRegressor

br = RandomForestRegressor()

br.fit(X_train, y_train)
y_pred = br.predict(X_test)

get_score(y_pred, y_test, 'bagging = RandomForestRegressor')

bagging = RandomForestRegressor
mse = 2213.06
r2_score = -0.33


# Бустинг (максимум 3 балла)

## Бустинг своими руками (2 балла)

Решите задачу при помощи алгоритма бустинга, используя в качестве базовой модели:
- Линейную регрессию
- Дерево
- Случайный лес

Какая модель дает лучший результат? Попробуйте улучшить его, перебрав несколько гиперпараметров (как у базовой модели, так и у ансамбля).

In [162]:
class Loss(ABC):
    """
    Базовый класс для функции потерь
    """
    @abstractmethod
    def forward(self, y_true: pd.Series, y_pred: pd.Series) -> float:
        """
        Метод, вычисляющий значение функции потерь
        """
        pass

    @abstractmethod
    def backward(self, y_true: pd.Series, y_pred: pd.Series) -> pd.Series:
        """
        Метод, вычисляющий значение градиента функции потерь по предсказаниям модели
        """
        pass

In [163]:
class MSELoss(Loss):
    def forward(self, y_pred: pd.Series, y_true: pd.Series) -> float:  # посчитаем значение ошибки
        return ((y_pred - y_true) ** 2).mean()

    def backward(self, y_pred: pd.Series, y_true: pd.Series) -> pd.Series:  # посчитаем производную по выходам модели
        return y_true - y_pred

In [172]:
class GradientBoostingRegressionSolver:
    def __init__(
        self,
        base_estimator_ctor,
        n_estimators: int = 10,
        loss: Loss = MSELoss(),
        learning_rate: float = 0.1,
        early_stopping: int = 10,
        **model_kwargs
    ):
        if early_stopping < 0:
            raise ValueError

        self._ctor = base_estimator_ctor
        self._kwargs = model_kwargs
        self._n_estimators = n_estimators
        self._estimators = []
        self._early_stopping = early_stopping
        self._loss = loss
        self._lr = learning_rate
        self._random_state = 42

    def _sample_data(self, X: pd.DataFrame, y: pd.Series, frac: float) -> Tuple[pd.DataFrame, pd.Series]:
        x_sample = X.sample(frac=frac, random_state=self._random_state)
        y_sample = y.loc[x_sample.index]
        return x_sample, y_sample

    def _split_data(
            self, X: pd.DataFrame, y: pd.Series, val_size: float
    ) -> Tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
        x_val, y_val = self._sample_data(X, y, val_size)
        x_train, y_train = X[~X.index.isin(x_val.index)], y[~y.index.isin(y_val.index)]
        return x_train, x_val, y_train, y_val

    def predict(self, X: pd.DataFrame) -> pd.Series:
        '''Прогоним данные, поступившие на вход, через все модели в ансамбле и сложим ответы '''
        pred = map(lambda x: x.predict(X), self._estimators)
        return sum(pred)

    def fit(self, X: pd.DataFrame, y: pd.Series, val_size: float = 0.1):
        '''Обучение'''
        # Хотим получить валидационную выборку, не тратя на это время снаружи
        x_train, x_val, y_train, y_val = self._split_data(X, y, val_size)
        # Создадим и обучим базовую модель
        base_estimator = DummyRegressor()
        base_estimator.fit(x_train, y_train)
        # Добавим базовую модель в список моделей
        self._estimators.append(base_estimator)

        # Посчитаем предсказания как на обучающей, так и на валидационной выборках
        y_pred_train, y_pred_val = self.predict(x_train), self.predict(x_val)

        # Посчитаем значение функции потерь для обучения и валидации
        train_loss, val_loss = self._loss.forward(y_train, y_pred_train), self._loss.forward(y_val, y_pred_val)

        # Посчитаем остатки, используя градиент функции потерь
        residuals = - self._lr * self._loss.backward(y_train, y_pred_train)

        print(f'train loss: {train_loss}, val loss: {val_loss}')

        previous_val_loss, cnt = val_loss, 0
        for i in range(self._n_estimators - 1):
            # Создадим очередную модель
            estimator = self._ctor(**self._kwargs)

            # 1. Обучим её и добавим в список моделей

            estimator.fit(x_train, residuals)
            self._estimators.append(estimator)

            # 2. Предскажем ВСЕМ ансамблем данные из обучающей выборки, то же самое сделаем для валидационной
            y_pred_train, y_pred_val = self.predict(x_train), self.predict(x_val)

            # 3. Посчитаем значения функции потерь (на обучении и валидации)
            train_loss, val_loss = self._loss.forward(y_train, y_pred_train), self._loss.forward(y_val, y_pred_val)

            # 4. Обновим остатки для обучающей выборки
            cnt += 1
            residuals = - self._lr * self._loss.backward(y_train, y_pred_train)

            print(f'train loss: {train_loss}, val loss: {val_loss}')
            # Если валидационный лосс несколько (self._early_stopping) шагов подряд не уменьшается, то остановим обучение

            if cnt > self._early_stopping:
                break

Вопросы на дополнительный балл:
- *Почему градиент по ответам мы берем со знаком минус?* Вектор градиента направлен в сторону увеличения функции,
мы используем знак минус что бы двигаться (в обратном направлении) к минимуму функции.
- *Почему в обучении мы домножаем на `learning_rate`, а в предсказаниях этого не делаем?*
learning_rate - шаг градиентного спуска, он выполняется на этапе обучения модели, мы умножаем на l_r вектор производных, на этапе предсказания в этом нет необходимости, тк моделька уже обучена, веса подобраны, градиентный спуск уже не выполняется

In [176]:
gbr = GradientBoostingRegressionSolver(LinearRegression, early_stopping=10)

gbr.fit(X_train, y_train)

y_pred = gbr.predict(X_test)

get_score(y_pred, y_test)

train loss: 2659.39066204753, val loss: 1850.9158807243762
train loss: 2650.796074752804, val loss: 1841.1717048106123
train loss: 2643.8344590440793, val loss: 1833.1478086897025
train loss: 2638.195550320011, val loss: 1826.530450564081
train loss: 2633.628034253516, val loss: 1821.0641884414151
train loss: 2629.92834623966, val loss: 1816.5409342852324
train loss: 2626.9315989484307, val loss: 1812.7910747655812
train loss: 2624.5042336425304, val loss: 1809.676267266842
train loss: 2622.53806774476, val loss: 1807.0835940338145
train loss: 2620.9454733675507, val loss: 1804.9208174719267
mse = 1651.89
r2_score = 0.01


# Catboost (1 балл)

Решите эту же задачу при помощи `catboost`, не перебирая гиперпараметры. Насколько лучше или хуже справился катбуст? В качестве эксперимента также попробуйте закинуть в него данные без предобработки (разумеется, выкинув ненужные колонки). Изменилось ли качество? Каким образом?

In [177]:
from catboost import CatBoostRegressor

## С использованием предобработанных данных

In [181]:
catboosting = CatBoostRegressor(random_state=42, metric_period=300)
catboosting.fit(X_train, y_train)
y_pred = catboosting.predict(X_test)

get_score(y_pred, y_test)

Learning rate set to 0.05298
0:	learn: 50.5614111	total: 1.86ms	remaining: 1.86s
300:	learn: 37.2372714	total: 383ms	remaining: 890ms
600:	learn: 32.0892930	total: 831ms	remaining: 552ms
900:	learn: 29.3498719	total: 1.25s	remaining: 137ms
999:	learn: 28.5364375	total: 1.36s	remaining: 0us
mse = 5754.71
r2_score = -2.45


## С использованием необработанных данных

In [201]:
df = pd.read_csv('data.csv').drop(columns=[
    'product_name',
    'index',
    'uniq_id',
    'customers_who_bought_this_item_also_bought',
    'items_customers_buy_after_viewing_this_item',
    'sellers',
    'description', # text
    'product_information', # text
    'product_description', # text
    'customer_questions_and_answers', # text
    'customer_reviews', # text
])
df['price'] = df['price'].str.replace('£', '')
df['price'] = df['price'].str.replace(',', '')
df['price'] = df['price'].astype('float')
df = df.dropna()

In [202]:
X, y = df.drop(columns=['price']), df.price
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [203]:
catboosting = CatBoostRegressor(
    random_state=42,
    metric_period=300,
    cat_features=[
        'manufacturer', 'number_available_in_stock', 'number_of_reviews',
        'average_review_rating', 'amazon_category_and_sub_category'
    ]
)
catboosting.fit(X_train, y_train)
y_pred = catboosting.predict(X_test)

get_score(y_pred, y_test)

Learning rate set to 0.05172
0:	learn: 37.3666326	total: 2.58ms	remaining: 2.57s
300:	learn: 29.3465994	total: 853ms	remaining: 1.98s
600:	learn: 26.7843836	total: 2.34s	remaining: 1.55s
900:	learn: 24.8630250	total: 3.3s	remaining: 363ms
999:	learn: 24.4064804	total: 3.66s	remaining: 0us
mse = 6325.02
r2_score = -0.02


Результаты CatBoostRegressor для данных без предобработки оказались хуже, чем для предобработанных данных,
также хочу заметить что в обоих случаях CatBoostRegressor работал с гиперпараетрами по умолчанию, и показал результаты хуже чем самописный бустинг

*Насколько лучше или хуже справился катбуст?* Катбуст справился хуже, я думаю это связвно с тем, что в самописном бустинге я использовал подобранные гиперпараметры (хочется в это верить)

## Комментарий
В любом случае результаты моделей полученные в этой домашке ужасные, r_2 ~ 2% такое я вижу в первый раз, я думаю что это связано с малым объёмом данных